# A Case Study: The Effect of Gun Ownership on Gun-Homicide Rates

We consider the problem of estimating the effect of gun
ownership on the homicide rate. For this purpose, we estimate the following partially
linear model

$$
 Y_{j,t} = \beta D_{j,(t-1)} + g(Z_{j,t}) + \epsilon_{j,t}.
$$

## Data

$Y_{j,t}$ is log homicide rate in county $j$ at time $t$, $D_{j, t-1}$ is log  fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership,  and  $Z_{j,t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. The parameter $\beta$ is the effect of gun ownership on the
homicide rates, controlling for county-level demographic and economic characteristics. 

The sample covers 195 large United States counties between the years 1980 through 1999, giving us 3900 observations.

In [1]:
using Pkg
Pkg.add("CSV"), using CSV
Pkg.add("DataFrames"), using DataFrames
Pkg.add("StatsModels"), using StatsModels
Pkg.add("GLM"), using GLM


    Updating registry at `C:\Users\PC\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\PC\.julia\environments\v1.6\Manifest.toml`


(nothing, nothing)

In [2]:
data = CSV.File("../data/gun_clean.csv") |> DataFrame;
println("Number of rows: ",size(data,1))
println("Number of columns: ",size(data,2))

Number of rows: 3900
Number of columns: 415


### Preprocessing

To account for heterogeneity across counties and time trends in  all variables, we remove from them county-specific and time-specific effects in the following preprocessing.

In [3]:
function varlist(df = nothing , type_dataframe = ["numeric","categorical","string"], pattern=String , exclude =  nothing)

    varrs = []
    if "numeric" in type_dataframe
        append!(varrs, [i for i in names(data) if eltype(eachcol(data)[i]) <: Number])    
    end
    if "categorical" in type_dataframe
        append!(varrs,[i for i in names(data) if eltype(eachcol(data)[i]) <: CategoricalVector])
    end
    if "string" in type_dataframe
        append!(varrs,[i for i in names(data) if eltype(eachcol(data)[i]) <: String])
    end
    varrs[(!varrs in exclude) & varrs[findall(x->contains(x,pattern),names(data))]]
end

varlist (generic function with 5 methods)

In [4]:
fixed = filter(x->contains(x,"X_Jfips"),names(data));
year = filter(x->contains(x,"X_Tyear"),names(data));

In [5]:
size(year)

(21,)

In [6]:
census = []
census_var = ["AGE", "BN", "BP", "BZ", "ED", "EL","HI", "HS", "INC", "LF", "LN", "PI", "PO", "PP", "PV", "SPR", "VS"]
for i in 1:size(census_var,1) 
    append!(census,filter(x->contains(x,census_var[i]),names(data)))
end

In [7]:
d = ["logfssl"]
y = ["logghomr"]
X1 = ["logrobr", "logburg", "burg_missing", "robrate_missing"]
X2 = ["newblack", "newfhh", "newmove", "newdens", "newmal"]

variable = [y, d,X1, X2, census]
varlis = []
for i in variable
    append!(varlis,i)
end


In [8]:
example = DataFrame(CountyCode = data[:,"CountyCode"]);
rdata = DataFrame(CountyCode = data[:,"CountyCode"]);

for i in 1:size(varlis,1)
    rdata[!,varlis[i]]= residuals(lm(term(Symbol(varlis[i])) ~ sum(term.(Symbol.(year))) + sum(term.(Symbol.(fixed))), data))
end

In [9]:
size(rdata)

(3900, 198)

Now, we can construct the treatment variable, the outcome variable and the matrix $Z$ that includes the control variables.

In [44]:
# treatment variable

D = rdata[!,d]
# Outcome variable
Y = rdata[!,y];

# Construct matrix Z
Z = rdata[!, varlis[3:end]]

size(Z)

(3900, 195)

In [45]:
typeof(D)

DataFrame

We have in total 195 control variables. The control variables $Z_{j,t}$ are from the U.S. Census Bureau and  contain demographic and economic characteristics of the counties such as  the age distribution, the income distribution, crime rates, federal spending, home ownership rates, house prices, educational attainment, voting paterns, employment statistics, and migration rates. 

In [48]:
clu = select(rdata,:CountyCode)
data = hcat(Y,D,Z,clu);

In [49]:
size(data)

(3900, 198)

## The effect of gun ownership


### OLS

After preprocessing the data, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls as a baseline model.

In [50]:
baseline_formula = @formula(logghomr ~ logfssl)
est_baseline = lm(baseline_formula, data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

logghomr ~ 1 + logfssl

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  -1.88455e-15  0.00672374  -0.00    1.0000  -0.0131824  0.0131824
logfssl       0.282304     0.057278     4.93    <1e-06   0.170007   0.394602
─────────────────────────────────────────────────────────────────────────────